In [1]:
!pip install transformers==4.15.0 sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', model_max_length=512)
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/json/tokenizer_json_15k.pkl', 'rb'))

In [4]:
vocab_dict = {}
context_preprocessed = []
misspell = []
misspell_type = []
misspell_word = []
with open('drive/MyDrive/AIBuilders/error.json') as f:
  contents = f.read()
  a = contents.split("\n")
  a = [j for j in a if not "$" in j]
  a = [j for j in a if not "^" in j]
  lena = len(a)
  for b in tqdm(range(10576)): #each sample
  # for b in tqdm(range(2880)): #each sample
    # print(f"================{b}==================")
    if "[]" in a[b]:
      a[b] = a[b].replace("[]", '["_"]')
    if '[" "]' in a[b]:
      a[b] = a[b].replace('[" "]', '["_"]')
    msp_in_sample = []
    a[b] = a[b].replace('"', "")
    a[b] = a[b][2:-2]
    part_2 = a[b].split("], [[[")
    sent = part_2[0].split("], [")[1]
    sent = sent.split(", ")
    # print(f"SENT: {sent}")
    msp = part_2[1].split("], [[")
    lenmsp = len(msp)
    msp_type_list = []
    for j in range(lenmsp):
      msp_type = msp[j].split("], ")[-1]
      msp_type = msp_type.replace("]", "")
      msp_type_list.append(msp_type)
      msp[j] = msp[j].split("], ")[:-1]
      lenmspj = len(msp[j])
      for k in range(lenmspj):
        msp[j][k] = msp[j][k].replace("[", "")
        msp[j][k] = msp[j][k].replace("]", "")
        msp[j][k] = msp[j][k].split(", ")
      msp[j][0][0] = int(msp[j][0][0])
      msp[j][0][1] = int(msp[j][0][1])
      # print(f"MSP: {msp[j]}, TYPE: {msp_type}")

    msp_word_list = []
    misspell_type.append(msp_type_list)

    for msp_ in msp:
      # print(msp_)
      st = msp_[0][0]
      ed = msp_[0][1]
      msp_arr = msp_[1]
      msp_arr = [k for k in msp_arr if not k == " " and not k == ""]
      # print(msp_arr)
      lenmsparr = len(msp_arr)
      if ed - st > lenmsparr:
        sent[st] = "^" + sent[st]
        sent[ed-1] = sent[ed-1] + "$"
        msp_in_sample.append("".join(msp_arr))
        correct = "".join(msp_arr)
        wrong = "".join(sent[st:ed])
        wrong = wrong.replace("^", "")
        wrong = wrong.replace("$", "")
        msp_word_list.append([(wrong, correct)])
      elif ed - st == lenmsparr:
        wrong_correct = []
        for it in range(st, ed):
          sent[it] = "^" + sent[it] + "$"
          msp_in_sample.append(msp_arr[it-st])
          correct = msp_arr[it-st]
          wrong = sent[it]
          wrong = wrong.replace("^", "")
          wrong = wrong.replace("$", "")
          wrong_correct.append((wrong, correct))
        msp_word_list.append(wrong_correct)
      elif ed - st < lenmsparr:
        wrong_correct = []
        for it in range(st, ed-1):
          sent[it] = "^" + sent[it] + "$"
          msp_in_sample.append(msp_arr[it-st])
          correct = msp_arr[it-st]
          wrong = sent[it]
          wrong = wrong.replace("^", "")
          wrong = wrong.replace("$", "")
          wrong_correct.append((wrong, correct))
        msp_word_list.append(wrong_correct)
        sent[ed-1] = "^" + sent[ed-1] + "$"
        msp_in_sample.append("".join(msp_arr[ed-1-st:]))

    context_preprocessed.append("".join(sent))

    misspell_word.append(msp_word_list)

    # print(sent)
    # print(msp_in_sample)
    # print(msp_type_list)
    # print(msp_word_list)
    # print("============================")
    misspell.append(msp_in_sample)

    for word in sent:
      if "$" not in word and "^" not in word:
        # print(word)
        if word not in vocab_dict:
          vocab_dict[word] = 0
        else:
          vocab_dict[word] += 1

    for word in msp_in_sample:
      if "$" not in word and "^" not in word:
        # print(word)
        if word not in vocab_dict:
          vocab_dict[word] = 0
        else:
          vocab_dict[word] += 1


vocab_dict = dict(sorted(vocab_dict.items(), key=lambda item: item[1], reverse=True))

100%|██████████| 10576/10576 [00:00<00:00, 32234.20it/s]


In [5]:
len(vocab_dict)

1684

In [6]:
vocab_dict

{' ': 7865,
 'ได้': 3433,
 'ค่ะ': 3162,
 'ครับ': 3154,
 'ไม่': 2943,
 'คะ': 2711,
 'อะ': 2035,
 'แล้ว': 1956,
 'ไหม': 1730,
 'เบอร์': 1698,
 'จะ': 1653,
 'ทำ': 1603,
 'สมัคร': 1473,
 'ที่': 1377,
 'ไป': 1309,
 'บัตร': 1259,
 'นี้': 1235,
 'เงิน': 1224,
 'ต้อง': 1177,
 'มี': 1155,
 'ใช้': 1123,
 'เปลี่ยน': 1073,
 'แต่': 1019,
 'ใหม่': 966,
 'ผม': 945,
 'โอน': 893,
 'เข้า': 873,
 'เลย': 866,
 'ก็': 863,
 'บัญชี': 853,
 'ว่า': 825,
 'มา': 752,
 'เป็น': 734,
 'ถ้า': 694,
 'ใช่': 647,
 'ให้': 621,
 'มัน': 610,
 'ใน': 603,
 '.': 595,
 'แอป': 586,
 'แบบ': 569,
 'อยาก': 567,
 'คือ': 550,
 'เรา': 544,
 'อย่างไร': 539,
 'นะ': 538,
 'ธนาคาร': 514,
 'พอดี': 486,
 'โทรศัพท์': 483,
 'ยัง': 482,
 'อยู่': 454,
 'เครื่อง': 439,
 'ของ': 431,
 'กับ': 416,
 'ไหน': 415,
 'ผ่าน': 405,
 'อะไร': 399,
 'ๆ': 397,
 'กด': 385,
 'สอบถาม': 368,
 'ขอบคุณ': 357,
 'สามารถ': 355,
 'ตู้': 354,
 'ตอน': 350,
 'เดิม': 340,
 'อีก': 333,
 'ยังไง': 328,
 'การ': 323,
 'ขอ': 318,
 'อ๋อ': 315,
 'วัน': 313,
 'หน่อย': 300,
 'กสิกร

In [7]:
print(vocab_dict)

{' ': 7865, 'ได้': 3433, 'ค่ะ': 3162, 'ครับ': 3154, 'ไม่': 2943, 'คะ': 2711, 'อะ': 2035, 'แล้ว': 1956, 'ไหม': 1730, 'เบอร์': 1698, 'จะ': 1653, 'ทำ': 1603, 'สมัคร': 1473, 'ที่': 1377, 'ไป': 1309, 'บัตร': 1259, 'นี้': 1235, 'เงิน': 1224, 'ต้อง': 1177, 'มี': 1155, 'ใช้': 1123, 'เปลี่ยน': 1073, 'แต่': 1019, 'ใหม่': 966, 'ผม': 945, 'โอน': 893, 'เข้า': 873, 'เลย': 866, 'ก็': 863, 'บัญชี': 853, 'ว่า': 825, 'มา': 752, 'เป็น': 734, 'ถ้า': 694, 'ใช่': 647, 'ให้': 621, 'มัน': 610, 'ใน': 603, '.': 595, 'แอป': 586, 'แบบ': 569, 'อยาก': 567, 'คือ': 550, 'เรา': 544, 'อย่างไร': 539, 'นะ': 538, 'ธนาคาร': 514, 'พอดี': 486, 'โทรศัพท์': 483, 'ยัง': 482, 'อยู่': 454, 'เครื่อง': 439, 'ของ': 431, 'กับ': 416, 'ไหน': 415, 'ผ่าน': 405, 'อะไร': 399, 'ๆ': 397, 'กด': 385, 'สอบถาม': 368, 'ขอบคุณ': 357, 'สามารถ': 355, 'ตู้': 354, 'ตอน': 350, 'เดิม': 340, 'อีก': 333, 'ยังไง': 328, 'การ': 323, 'ขอ': 318, 'อ๋อ': 315, 'วัน': 313, 'หน่อย': 300, 'กสิกร': 298, 'แอพ': 297, 'มือถือ': 296, 'ด้วย': 293, 'ไง': 291, 'แจ้ง': 288, 

In [8]:
ds_tag = pickle.load(open('drive/MyDrive/AIBuilders/json/ner_json_15k_2.pkl', 'rb'))
ds_mlm = pickle.load(open('drive/MyDrive/AIBuilders/json/mlm_json_15k_2.pkl', 'rb'))

In [9]:
ds_mlm = ds_mlm[10576:]
ds_tag = ds_tag[10576:]
NUM_SAMPLE = ds_mlm.shape[0]
NUM_SAMPLE

5000

In [10]:
for i in tqdm(range(NUM_SAMPLE)):
  sent_msp = ds_tag.iloc[i]['text']['input_ids']
  sent_crt = ds_mlm.iloc[i]['labels']
  sent_msp = [int(j) for j in sent_msp[0] if j not in [1, 5, 6]]
  sent_crt = [j for j in sent_crt if j not in [1, 5, 6]]  
  sent_msp = tokenizer.convert_ids_to_tokens(sent_msp)
  sent_crt = tokenizer.convert_ids_to_tokens(sent_crt)
  # print(sent_msp)
  # print(sent_crt)
  for word in sent_msp:
    if word not in vocab_dict:
      vocab_dict[word] = 0
    else:
      vocab_dict[word] += 1
  for word in sent_crt:
    if word not in vocab_dict:
      vocab_dict[word] = 0
    else:
      vocab_dict[word] += 1

100%|██████████| 5000/5000 [00:14<00:00, 337.50it/s]


In [11]:
len(vocab_dict)

4005

In [12]:
cou = 0
for word in vocab_dict:
  vocab_dict[word] = cou
  cou += 1

In [13]:
for key in vocab_dict:
  print(f"{key}: {vocab_dict[key]}")

 : 0
ได้: 1
ค่ะ: 2
ครับ: 3
ไม่: 4
คะ: 5
อะ: 6
แล้ว: 7
ไหม: 8
เบอร์: 9
จะ: 10
ทำ: 11
สมัคร: 12
ที่: 13
ไป: 14
บัตร: 15
นี้: 16
เงิน: 17
ต้อง: 18
มี: 19
ใช้: 20
เปลี่ยน: 21
แต่: 22
ใหม่: 23
ผม: 24
โอน: 25
เข้า: 26
เลย: 27
ก็: 28
บัญชี: 29
ว่า: 30
มา: 31
เป็น: 32
ถ้า: 33
ใช่: 34
ให้: 35
มัน: 36
ใน: 37
.: 38
แอป: 39
แบบ: 40
อยาก: 41
คือ: 42
เรา: 43
อย่างไร: 44
นะ: 45
ธนาคาร: 46
พอดี: 47
โทรศัพท์: 48
ยัง: 49
อยู่: 50
เครื่อง: 51
ของ: 52
กับ: 53
ไหน: 54
ผ่าน: 55
อะไร: 56
ๆ: 57
กด: 58
สอบถาม: 59
ขอบคุณ: 60
สามารถ: 61
ตู้: 62
ตอน: 63
เดิม: 64
อีก: 65
ยังไง: 66
การ: 67
ขอ: 68
อ๋อ: 69
วัน: 70
หน่อย: 71
กสิกร: 72
แอพ: 73
มือถือ: 74
ด้วย: 75
ไง: 76
แจ้ง: 77
หรือ: 78
ทาง: 79
ขึ้น: 80
โทร: 81
ทำไม: 82
บอก: 83
ยกเลิก: 84
บ้าง: 85
เครดิต: 86
ทราบ: 87
มาก: 88
รหัส: 89
เสีย: 90
รู้: 91
เก่า: 92
ติดต่อ: 93
เขา: 94
เคย: 95
ไว้: 96
ถึง: 97
บริการ: 98
พร้อมเพย์: 99
หาย: 100
เปิด: 101
จาก: 102
พอ: 103
เอา: 104
อ่ะ: 105
ค่า: 106
ออก: 107
เพราะ: 108
ผิด: 109
สาขา: 110
?: 111
ระบบ: 112
คน: 113
ดี: 114
ยอด: 115


In [14]:
len(tokenizer)

25354

In [15]:
embeds = nn.Embedding(1684, 64)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([vocab_dict["ครับ"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[-0.7395, -0.0389,  2.8052,  0.7514, -1.0470,  1.0306, -0.3010, -0.9940,
         -0.8083, -0.1963, -2.2851, -0.6918, -0.8492, -0.3294,  2.6770,  2.5288,
          0.0392,  0.7233, -1.6527,  0.4845, -0.4365, -1.1028,  0.6069, -0.3300,
          0.1208,  0.1187,  0.6011,  0.0619,  0.6992,  2.8214, -0.6539,  0.4439,
         -0.1081, -0.2425,  0.6679, -1.7700,  1.8445,  0.2722, -0.3426,  0.6390,
          1.8588, -0.4709,  2.1204,  0.1433, -2.6522,  2.8603, -1.2139,  0.7065,
         -1.4903, -0.6434,  0.3752, -0.5868,  1.4225,  0.9710,  0.9772,  1.6954,
          0.5201,  1.2563,  0.3342,  0.9279, -0.7981,  1.4075,  0.1050,  1.0328]],
       grad_fn=<EmbeddingBackward0>)


In [16]:
class DetectorModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(DetectorModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim * context_size, hidden_size=64,
                            num_layers=2, bidirectional=True)
        self.linear = nn.Linear(embedding_dim*2, 512)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        # out = F.relu(self.linear1(embeds))
        out, (hidden, cell) = self.lstm(embeds)
        out = self.linear(out)
        probs = self.softmax(out)
        print(probs)
        return probs

In [17]:
char_vocab = {' ': 2, '!': 5, '"': 25, '#': 4, '$': 3, '%': 6, '&': 7, "'": 24, '(': 26, ')': 27, '*': 8, '+': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 66, '1': 205, '2': 154, '3': 265, '4': 200, '5': 151, '6': 275, '7': 162, '8': 72, '9': 354, ':': 14, ';': 15, '<': 30, '=': 21, '>': 31, '?': 16, '@': 17, '[': 32, ']': 33, '^': 18, '_': 19, '`': 20, 'a': 196, 'b': 194, 'c': 339, 'd': 355, 'e': 300, 'f': 350, 'g': 19, 'h': 83, 'i': 227, 'j': 248, 'k': 144, 'l': 199, 'm': 284, 'n': 234, 'o': 44, 'p': 228, 'pad': 0, 'q': 360, 'r': 34, 's': 270, 't': 169, 'u': 40, 'unknown': 1, 'v': 215, 'w': 195, 'x': 26, 'y': 101, 'z': 78, '{': 28, '|': 22, '}': 29, '~': 23, '\x93': 246, '\x94': 263, '\xa0': 59, '¡': 190, '£': 61, '§': 38, '«': 70, '\xad': 45, '®': 99, '°': 347, '±': 22, '²': 150, '³': 288, '´': 329, 'µ': 206, '·': 173, '¹': 90, '»': 326, '¼': 28, '½': 41, '¾': 239, '×': 112, 'ß': 315, 'à': 43, 'á': 266, 'â': 91, 'ã': 25, 'ä': 289, 'å': 54, 'æ': 211, 'ç': 24, 'è': 338, 'é': 340, 'ê': 21, 'ë': 6, 'ì': 133, 'í': 108, 'î': 223, 'ï': 37, 'ð': 258, 'ñ': 175, 'ò': 9, 'ó': 132, 'ô': 212, 'ö': 60, '÷': 136, 'ø': 302, 'ù': 229, 'ú': 179, 'û': 249, 'ü': 296, 'ý': 75, 'þ': 216, 'ā': 264, 'ă': 349, 'ą': 322, 'č': 279, 'đ': 277, 'ē': 183, 'ę': 335, 'ě': 317, 'ğ': 186, 'ī': 276, 'ı': 318, 'ł': 139, 'ń': 74, 'ŋ': 257, 'ō': 122, 'ŏ': 107, 'ő': 180, 'œ': 67, 'ś': 123, 'ş': 224, 'š': 97, 'ū': 79, 'ŭ': 226, 'ž': 182, 'ơ': 64, 'ư': 76, 'ǎ': 98, 'ǐ': 159, 'ǒ': 198, 'ǔ': 233, 'ɔ': 331, 'ə': 116, 'ɛ': 104, 'ɪ': 165, 'ʃ': 368, 'ʒ': 102, 'ʰ': 220, 'ʿ': 312, 'ˈ': 362, 'ː': 63, '̇': 307, '̍': 292, '̤': 321, '̪': 168, 'α': 303, 'β': 359, 'γ': 236, 'δ': 311, 'ε': 305, 'θ': 304, 'κ': 115, 'λ': 346, 'μ': 27, 'π': 52, 'σ': 313, 'φ': 138, 'χ': 343, 'ψ': 185, 'ω': 278, 'в': 85, 'г': 176, 'ж': 157, 'и': 55, 'й': 214, 'к': 17, 'с': 32, 'ч': 333, 'ш': 46, 'щ': 314, 'ы': 310, 'ь': 92, 'э': 256, 'я': 147, 'ё': 71, 'ө': 204, 'إ': 282, 'ا': 218, 'ب': 201, 'ج': 20, 'ع': 251, 'ف': 301, 'ق': 50, 'ل': 155, 'م': 323, 'ن': 334, 'ه': 110, 'و': 164, 'ي': 178, 'َ': 352, 'ُ': 130, 'ِ': 31, 'ّ': 117, 'ْ': 273, 'द': 89, 'न': 262, 'म': 113, 'व': 268, 'ी': 267, 'े': 306, 'ॐ': 351, 'ก': 342, 'ข': 77, 'ฃ': 36, 'ค': 191, 'ฅ': 167, 'ฆ': 118, 'ง': 163, 'จ': 125, 'ฉ': 260, 'ช': 344, 'ซ': 166, 'ฌ': 353, 'ญ': 308, 'ฎ': 131, 'ฏ': 53, 'ฐ': 135, 'ฑ': 187, 'ฒ': 259, 'ณ': 327, 'ด': 121, 'ต': 126, 'ถ': 15, 'ท': 95, 'ธ': 156, 'น': 73, 'บ': 16, 'ป': 189, 'ผ': 369, 'ฝ': 7, 'พ': 49, 'ฟ': 103, 'ภ': 160, 'ม': 293, 'ย': 243, 'ร': 11, 'ฤ': 320, 'ล': 366, 'ฦ': 325, 'ว': 119, 'ศ': 23, 'ษ': 106, 'ส': 177, 'ห': 274, 'ฬ': 153, 'อ': 235, 'ฮ': 294, 'ฯ': 29, 'ะ': 170, 'ั': 82, 'า': 88, 'ำ': 127, 'ิ': 62, 'ี': 272, 'ึ': 287, 'ื': 111, 'ุ': 5, 'ู': 184, 'ฺ': 161, 'เ': 330, 'แ': 309, 'โ': 337, 'ใ': 232, 'ไ': 210, 'ๅ': 356, 'ๆ': 319, '็': 363, '่': 269, '้': 247, '๊': 158, '๋': 152, '์': 142, 'ํ': 230, '๐': 120, '๑': 143, '๒': 124, '๓': 345, '๔': 8, '๕': 213, '๖': 221, '๗': 13, '๘': 33, '๙': 357, 'ḍ': 271, 'ḥ': 39, 'ṃ': 324, 'ṅ': 255, 'ṇ': 69, 'ṛ': 56, 'ṣ': 286, 'ṭ': 238, 'ạ': 244, 'ả': 250, 'ầ': 252, 'ế': 51, 'ễ': 134, 'ệ': 93, 'ị': 129, 'ọ': 219, 'ộ': 100, 'ỽ': 47, '\u2003': 140, '\u2005': 58, '\u200b': 348, '\u200c': 193, '\u200d': 192, '\u200e': 149, '–': 171, '—': 109, '‘': 336, '’': 358, '“': 87, '”': 137, '†': 128, '‡': 48, '•': 328, '…': 12, '′': 316, '‹': 30, '›': 141, '※': 181, '⁄': 285, '€': 80, '℃': 237, '™': 241, '→': 146, '↔': 209, '∈': 81, '∑': 65, '−': 283, '∘': 172, '√': 96, '∞': 68, '∨': 298, '≈': 10, '≠': 105, '≡': 242, '≤': 197, '≥': 84, '⊕': 14, '⊙': 299, '⋯': 35, '▪': 57, '○': 148, '●': 245, '◦': 225, '☆': 364, '♠': 261, '♣': 295, '♥': 42, '♦': 281, '♪': 18, '♯': 94, '\u3000': 254, '。': 86, '《': 174, '》': 290, '【': 253, 'い': 332, 'う': 217, 'え': 280, 'か': 188, '・': 341, '叉': 114, '夜': 297, '奖': 367, '犬': 240, '获': 365, '虫': 207, '\uf022': 361, '\uf712': 203, '\ufeff': 202, '（': 231, '）': 222, '：': 208, '～': 145, '\U00100083': 291}

In [18]:
WORD_EMBEDDING_DIM = 64
CHAR_EMBEDDING_DIM = 128
MAX_WORD_IN_SENT = 512
MAX_CHAR_IN_WORD = 30
len(char_vocab)

399

In [22]:
class DetectorModeler(nn.Module):

    def __init__(self, word_vocab_size, word_embedding_dim, max_word_in_sent, char_vocab_size, char_embedding_dim, max_char_in_word):
        super(DetectorModeler, self).__init__()
        self.char_embeddings = nn.Embedding(char_vocab_size, char_embedding_dim)
        self.char_lstm = nn.LSTM(char_embedding_dim * max_word_in_sent * max_char_in_word, hidden_size=32,
                            num_layers=1, bidirectional=True)
        self.word_embeddings = nn.Embedding(word_vocab_size, word_embedding_dim)

        self.lstm = nn.LSTM((word_embedding_dim * max_word_in_sent) + (64), hidden_size=64,
                            num_layers=2, bidirectional=True)
        self.linear = nn.Linear(128, 512)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, word_in, char_in):
        char_embeds = self.char_embeddings(char_in).view((1, -1))
        char_enc, (hidden, cell) = self.char_lstm(char_embeds)
        word_embeds = self.word_embeddings(word_in).view((1, -1))
        # print(char_enc.size())
        # print(word_embeds.size())
        all_embeds = torch.cat((char_enc, word_embeds), 1)
        # print(all_embeds.size())
        out, (hidden, cell) = self.lstm(all_embeds)
        out = self.linear(out)
        # out = F.relu(self.linear1(embeds))
        probs = self.softmax(out)
        print(probs)
        return probs

In [23]:
losses = []
loss_function = nn.CrossEntropyLoss()
model = DetectorModeler(len(tokenizer), WORD_EMBEDDING_DIM, MAX_WORD_IN_SENT, len(char_vocab), CHAR_EMBEDDING_DIM, MAX_CHAR_IN_WORD)
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [33]:
for epoch in tqdm(range(10)):
    total_loss = 0
    for sent_id in range(10):
      text = ds_tag.iloc[sent_id]['text']['input_ids'][0].tolist()
      # print(text)
      # text = [k for k in text if k != 1]
      labels = ds_tag.iloc[sent_id]['labels'].tolist()
      # labels = [k for k in labels if k != 1]

      sent_len = len(text)
      char_in = []
      for i in range(sent_len):
        word = tokenizer.convert_ids_to_tokens(text[i])
        word = word.replace("▁", " ")
        # print(f"WORD: {word}")
        char_list = [char_vocab[j] for j in word]
        while len(char_list) < 30:
          char_list.append(0)
        # print(char_list)
        char_in.append(char_list)

      char_in = torch.tensor(char_in, dtype=torch.long)
      word_in = torch.tensor([text], dtype=torch.long)
      print(char_in.size())
      print(word_in.size())
      model.zero_grad()

      probs = model(word_in, char_in)
      print(probs.shape)
      labels = torch.tensor([labels], dtype=torch.float)
      print(labels.shape)

      loss = loss_function(probs, labels)

      loss.backward()
      optimizer.step()

      total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([512, 30])
torch.Size([1, 512])
tensor([[0.0028, 0.0037, 0.0036, 0.0039, 0.0047, 0.0042, 0.0032, 0.0029, 0.0035,
         0.0026, 0.0022, 0.0032, 0.0025, 0.0025, 0.0031, 0.0032, 0.0029, 0.0024,
         0.0019, 0.0019, 0.0018, 0.0022, 0.0021, 0.0016, 0.0022, 0.0021, 0.0017,
         0.0018, 0.0022, 0.0015, 0.0018, 0.0020, 0.0019, 0.0021, 0.0019, 0.0024,
         0.0018, 0.0020, 0.0016, 0.0019, 0.0020, 0.0023, 0.0016, 0.0021, 0.0017,
         0.0017, 0.0019, 0.0016, 0.0017, 0.0023, 0.0017, 0.0019, 0.0020, 0.0022,
         0.0024, 0.0017, 0.0020, 0.0018, 0.0017, 0.0020, 0.0019, 0.0020, 0.0023,
         0.0015, 0.0021, 0.0019, 0.0017, 0.0019, 0.0022, 0.0018, 0.0020, 0.0019,
         0.0016, 0.0018, 0.0015, 0.0019, 0.0023, 0.0019, 0.0018, 0.0019, 0.0017,
         0.0017, 0.0016, 0.0020, 0.0016, 0.0018, 0.0019, 0.0026, 0.0019, 0.0018,
         0.0024, 0.0022, 0.0018, 0.0019, 0.0017, 0.0019, 0.0020, 0.0019, 0.0019,
         0.0018, 0.0024, 0.0019, 0.0019, 0.0022, 0.0019, 0.0023, 0

 10%|█         | 1/10 [01:28<13:18, 88.74s/it]

torch.Size([512, 30])
torch.Size([1, 512])
tensor([[0.0223, 0.0381, 0.0341, 0.0402, 0.0596, 0.0418, 0.0242, 0.0167, 0.0148,
         0.0069, 0.0055, 0.0112, 0.0062, 0.0059, 0.0101, 0.0104, 0.0079, 0.0057,
         0.0013, 0.0012, 0.0014, 0.0018, 0.0017, 0.0010, 0.0015, 0.0014, 0.0011,
         0.0011, 0.0016, 0.0008, 0.0011, 0.0013, 0.0013, 0.0016, 0.0013, 0.0021,
         0.0010, 0.0015, 0.0009, 0.0012, 0.0015, 0.0016, 0.0009, 0.0015, 0.0010,
         0.0009, 0.0013, 0.0007, 0.0009, 0.0017, 0.0011, 0.0013, 0.0013, 0.0017,
         0.0021, 0.0009, 0.0014, 0.0013, 0.0010, 0.0016, 0.0012, 0.0013, 0.0019,
         0.0008, 0.0014, 0.0011, 0.0010, 0.0012, 0.0018, 0.0010, 0.0017, 0.0011,
         0.0010, 0.0011, 0.0008, 0.0014, 0.0023, 0.0013, 0.0012, 0.0012, 0.0012,
         0.0010, 0.0010, 0.0013, 0.0010, 0.0013, 0.0012, 0.0023, 0.0012, 0.0011,
         0.0019, 0.0017, 0.0011, 0.0012, 0.0009, 0.0013, 0.0018, 0.0013, 0.0014,
         0.0011, 0.0020, 0.0012, 0.0011, 0.0020, 0.0011, 0.0018, 0

 20%|██        | 2/10 [02:54<11:36, 87.12s/it]

...

torch.Size([512, 30])
torch.Size([1, 512])
tensor([[1.3737e-05, 4.3310e-05, 2.5841e-05, 4.2025e-05, 9.9981e-01, 3.6537e-05,
         1.9683e-05, 5.2870e-06, 1.2955e-06, 2.8169e-07, 2.0985e-07, 5.3349e-07,
         2.4190e-07, 2.1641e-07, 4.9739e-07, 5.0134e-07, 2.6733e-07, 2.0567e-07,
         1.4815e-08, 1.2815e-08, 1.7483e-08, 1.9964e-08, 6.9081e-09, 3.6769e-09,
         6.7448e-09, 6.3181e-09, 4.0355e-09, 4.7340e-09, 6.8181e-09, 2.8897e-09,
         3.8795e-09, 4.9373e-09, 6.2361e-09, 7.5560e-09, 4.9197e-09, 1.0943e-08,
         3.3619e-09, 6.2094e-09, 3.1919e-09, 5.0848e-09, 7.0116e-09, 6.0449e-09,
         2.6987e-09, 6.1195e-09, 3.9619e-09, 3.5112e-09, 5.1465e-09, 2.6258e-09,
         2.6159e-09, 7.9332e-09, 4.0014e-09, 4.6257e-09, 5.0032e-09, 6.4566e-09,
         1.0977e-08, 3.2733e-09, 5.7700e-09, 5.2719e-09, 3.6486e-09, 7.6492e-09,
         3.8023e-09, 4.6624e-09, 9.2776e-09, 2.9423e-09, 5.4987e-09, 3.6551e-09,
         3.5141e-09, 4.1849e-09, 7.7912e-09, 3.3001e-09, 8.4454e-0

 90%|█████████ | 9/10 [13:18<01:30, 90.95s/it]

torch.Size([512, 30])
torch.Size([1, 512])
tensor([[1.2681e-05, 3.9833e-05, 2.3834e-05, 3.8729e-05, 9.9982e-01, 3.3676e-05,
         1.8270e-05, 4.9468e-06, 1.2257e-06, 2.6647e-07, 1.9781e-07, 5.0479e-07,
         2.2892e-07, 2.0471e-07, 4.7020e-07, 4.7403e-07, 2.5243e-07, 1.9481e-07,
         1.4039e-08, 1.2148e-08, 1.6524e-08, 1.8935e-08, 6.5472e-09, 3.4809e-09,
         6.4057e-09, 6.0067e-09, 3.8285e-09, 4.4894e-09, 6.4495e-09, 2.7413e-09,
         3.6730e-09, 4.6637e-09, 5.9130e-09, 7.1747e-09, 4.6495e-09, 1.0378e-08,
         3.1878e-09, 5.8709e-09, 3.0198e-09, 4.8240e-09, 6.6426e-09, 5.7436e-09,
         2.5551e-09, 5.8019e-09, 3.7538e-09, 3.3281e-09, 4.8729e-09, 2.4830e-09,
         2.4756e-09, 7.5128e-09, 3.7959e-09, 4.3767e-09, 4.7333e-09, 6.1113e-09,
         1.0404e-08, 3.0922e-09, 5.4716e-09, 4.9889e-09, 3.4564e-09, 7.2637e-09,
         3.6028e-09, 4.4095e-09, 8.7791e-09, 2.7922e-09, 5.2193e-09, 3.4673e-09,
         3.3289e-09, 3.9693e-09, 7.3631e-09, 3.1288e-09, 7.9838e-0

100%|██████████| 10/10 [14:59<00:00, 89.92s/it]

[-3131002.34375, -3131584.75, -3132028.625, -3132479.75, -3132602.65625, -3132605.625, -3132606.59375, -3132606.59375, -3132606.6875, -3132606.8125]
